In [4]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import exists
import re

In [41]:
folderpath = "bestsellerdata"

data = pd.DataFrame()

for path in listdir(folderpath):
    if path.endswith(".xlsx"):
        weekMatch = re.search("(\d{2}-\d{2}-\d{2})", path)
        sheets = dict()
        filedata = pd.read_excel(f"{folderpath}/{path}", sheet_name=None)
        if weekMatch:
            week = weekMatch.group(1)
            sheets[week] = pd.DataFrame()
            for sheetname in filedata.keys():
                df = filedata[sheetname]
                for column in df.columns.values:
                    if column == "Info":
                        infoMatch = re.search("\w")
                        print(df.loc[1,column])
        elif re.search("(\d{2}-\d{2})", path):
            for sheetname in filedata.keys():
                sheets[sheetname] = filedata[sheetname] 

Wonder | R.J. Palacio $16.99  | ISBN: 9781524720193
The Immortalists | Chloe Benjamin $26.00  | ISBN: 9780735213180
Astrophysics for People in a Hurry | Neil deGrasse Tyson $18.95  | ISBN: 9780393609394
The Whistler | John Grisham $9.99  | ISBN: 9781101967683
Pachinko | Min Jin Lee $15.99  | ISBN: 9781455563920
The Silk Roads | Peter Frankopan $20.00  | ISBN: 9781101912379
Turtles All the Way Down | John Green $19.99  | ISBN: 9780525555360
The Underground Railroad | Colson Whitehead $16.95, ISBN: 9780345804327
The Silk Roads | Peter Frankopan $20.00 | ISBN: 9781101912379
The Great Alone | Kristin Hannah $28.99 | ISBN: 9780312577230
Astrophysics for People in a Hurry | Neil deGrasse Tyson $18.95 | ISBN: 9780393609394
Ready Player One | Ernest Cline $9.99 | ISBN: 9780804190145
Wonder | R.J. Palacio $16.99 | ISBN: 9781524720193
Turtles All the Way Down | John Green $19.99 | ISBN: 9780525555360
The Underground Railroad | Colson Whitehead $16.95 | ISBN: 9780345804327
The Stranger in the Woo